## Packages:


In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

!pip install ortools

     |████████████████████████████████| 27.9MB 145kB/s 
     |████████████████████████████████| 1.3MB 50.2MB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


## Web Scraping Functions:

https://www.formula1.com/en/results.html

In [0]:
def get_Drivers(year):
  page = requests.get("https://www.formula1.com/en/results.html/"+str(year)+"/drivers.html")
  soup = BeautifulSoup(page.content, 'html.parser')
  table_data = soup.select('.table-wrap')[0].text.split("\n\n\n\n\n")
  Drivers = list()
  for line in table_data[1:]:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 4:
      Drivers.append(temp[1]+" "+temp[2])
  return Drivers

In [3]:
# EXAMPLE:
yr = 2019
get_Drivers(yr)

['Lewis Hamilton',
 'Valtteri Bottas',
 'Max Verstappen',
 'Charles Leclerc',
 'Sebastian Vettel',
 'Carlos Sainz',
 'Pierre Gasly',
 'Alexander Albon',
 'Daniel Ricciardo',
 'Sergio Perez',
 'Lando Norris',
 'Kimi Räikkönen',
 'Daniil Kvyat',
 'Nico Hulkenberg',
 'Lance Stroll',
 'Kevin Magnussen',
 'Antonio Giovinazzi',
 'Romain Grosjean',
 'Robert Kubica',
 'George Russell']

In [0]:
def get_Teams(year):
  page = requests.get("https://www.formula1.com/en/results.html/"+str(year)+"/team.html")
  soup = BeautifulSoup(page.content, 'html.parser')
  table_data = soup.select('.table-wrap')[0].text.split("\n\n\n\n\n")
  Teams = list()
  for line in table_data[1:]:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 2:
      Teams.append(temp[1])
  return Teams

In [5]:
# EXAMPLE:
get_Teams(yr)

['Mercedes',
 'Ferrari',
 'Red Bull Racing Honda',
 'McLaren Renault',
 'Renault',
 'Scuderia Toro Rosso Honda',
 'Racing Point BWT Mercedes',
 'Alfa Romeo Racing Ferrari',
 'Haas Ferrari',
 'Williams Mercedes']

In [0]:
def get_Races(year):
  page = requests.get("https://www.formula1.com/en/results.html/"+str(year)+"/races.html")
  soup = BeautifulSoup(page.content, 'html.parser')
  table_data = soup.select('.table-wrap')[0].text.split("\n\n\n\n\n")
  Races = list()
  for line in table_data[2:]:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 5:
      Races.append(temp[0].strip().lower())
  return Races

In [7]:
# EXAMPLE:
get_Races(yr)

['australia',
 'bahrain',
 'china',
 'azerbaijan',
 'spain',
 'monaco',
 'canada',
 'france',
 'austria',
 'great britain',
 'germany',
 'hungary',
 'belgium',
 'italy',
 'singapore',
 'russia',
 'japan',
 'mexico',
 'united states',
 'brazil',
 'abu dhabi']

In [0]:
# Each race is associated with a ordered index after 2016...
def get_RaceIndex(year):
  Dict = {2020: '1021', 2019: '1000', 2018: '979', 2017: '959'}
  if year > max(Dict.keys()) or year < min(Dict.keys()):
    print("Manual Update Dictionary")
    return None
  else:
    start_index = int(Dict[year])
    Races = get_Races(year)
    Index = np.array(range(len(Races))) + start_index
    return dict(zip(Races,Index))

In [9]:
# EXAMPLE: https://www.formula1.com/en/results.html/2017/races/969/hungary/race-result.html <---- 969
get_RaceIndex(2017)['hungary']

969

In [0]:
def get_TableData(year,index_dict,race,table): 
  url = "https://www.formula1.com/en/results.html/"+str(year)+"/races/" + str(index_dict[race]) + "/" + str(race) + "/" + str(table) + ".html"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  table_data = soup.select('.resultsarchive-col-right')[0].text.split("\n\n\n\n\n")
  return table_data

In [11]:
# EXAMPLE: 
yr = 2018
r = "mexico"
i_d = get_RaceIndex(yr)
tables = ["qualifying", "starting-grid", "fastest-laps", "race-result"]

get_TableData(yr,i_d,r,tables[3]) 

['',
 'Pos\nNo\nDriver\nCar\nLaps\nTime/Retired\nPTS',
 '\n\n1\n33\n\nMax\nVerstappen\nVER\n\nRed Bull Racing TAG Heuer\n71\n1:38:28.851\n25',
 '2\n5\n\nSebastian\nVettel\nVET\n\nFerrari\n71\n+17.316s\n18',
 '3\n7\n\nKimi\nRäikkönen\nRAI\n\nFerrari\n71\n+49.914s\n15',
 '4\n44\n\nLewis\nHamilton\nHAM\n\nMercedes\n71\n+78.738s\n12',
 '5\n77\n\nValtteri\nBottas\nBOT\n\nMercedes\n70\n+1 lap\n10',
 '6\n27\n\nNico\nHulkenberg\nHUL\n\nRenault\n69\n+2 laps\n8',
 '7\n16\n\nCharles\nLeclerc\nLEC\n\nSauber Ferrari\n69\n+2 laps\n6',
 '8\n2\n\nStoffel\nVandoorne\nVAN\n\nMcLaren Renault\n69\n+2 laps\n4',
 '9\n9\n\nMarcus\nEricsson\nERI\n\nSauber Ferrari\n69\n+2 laps\n2',
 '10\n10\n\nPierre\nGasly\nGAS\n\nScuderia Toro Rosso Honda\n69\n+2 laps\n1',
 '11\n31\n\nEsteban\nOcon\nOCO\n\nForce India Mercedes\n69\n+2 laps\n0',
 '12\n18\n\nLance\nStroll\nSTR\n\nWilliams Mercedes\n69\n+2 laps\n0',
 '13\n35\n\nSergey\nSirotkin\nSIR\n\nWilliams Mercedes\n69\n+2 laps\n0',
 '14\n28\n\nBrendon\nHartley\nHAR\n\nScu

## Format Functions:

In [0]:
def format_Qualifying(table_data):
  #QUALIFYING: 
  Table = pd.DataFrame()
  Pos, No, Driver, Car, Q1, Q2, Q3 = list(), list(), list(), list(), list(),list(), list()

  for line in table_data[2:]:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 7:
      Pos.append(temp[0])
      No.append(temp[1]) 
      Driver.append(temp[2]+" "+temp[3])
      Car.append(temp[5])
      Q1.append(temp[6])
      if len(temp) > 8:
        Q2.append(temp[7])
      else:
        Q2.append(" ")
      if len(temp) > 9:
        Q3.append(temp[8])
      else:
        Q3.append(" ")

  Table["Pos"], Table["No"], Table["Driver"], Table["Car"], Table["Q1"], Table["Q2"], Table["Q3"] = Pos, No, Driver, Car, Q1, Q2, Q3
  return Table

In [13]:
# EXAMPLE:
td = get_TableData(yr,i_d,r,tables[0]) 
format_Qualifying(td)

,Pos,No,Driver,Car,Q1,Q2,Q3
0,1,3,Daniel Ricciardo,Red Bull Racing TAG Heuer,1:15.866,1:15.845,1:14.759
1,2,33,Max Verstappen,Red Bull Racing TAG Heuer,1:15.756,1:15.640,1:14.785
2,3,44,Lewis Hamilton,Mercedes,1:15.673,1:15.644,1:14.894
3,4,5,Sebastian Vettel,Ferrari,1:16.089,1:15.715,1:14.970
4,5,77,Valtteri Bottas,Mercedes,1:15.580,1:15.923,1:15.160
5,6,7,Kimi Räikkönen,Ferrari,1:16.446,1:15.996,1:15.330
6,7,27,Nico Hulkenberg,Renault,1:16.498,1:16.126,1:15.827
7,8,55,Carlos Sainz,Renault,1:16.813,1:16.188,1:16.084
8,9,16,Charles Leclerc,Sauber Ferrari,1:16.862,1:16.320,1:16.189
9,10,9,Marcus Ericsson,Sauber Ferrari,1:16.701,1:16.633,1:16.513


In [0]:
def format_StartingGrid(table_data):
  #STARTING GRID: 
  Table = pd.DataFrame()
  Pos, No, Driver, Car, Time = list(), list(), list(), list(), list()

  for line in table_data[2:]:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 5:
      Pos.append(temp[0])
      No.append(temp[1]) 
      Driver.append(temp[2]+" "+temp[3])
      Car.append(temp[5])
      try:
        Time.append(temp[6])
      except:
        Time.append("NT")

  Table["Pos"], Table["No"], Table["Driver"], Table["Car"], Table["Time"] = Pos, No, Driver, Car, Time
  return Table

In [15]:
# EXAMPLE:
td = get_TableData(yr,i_d,r,tables[1]) 
format_StartingGrid(td)

,Pos,No,Driver,Car,Time
0,1,3,Daniel Ricciardo,Red Bull Racing TAG Heuer,1:14.759
1,2,33,Max Verstappen,Red Bull Racing TAG Heuer,1:14.785
2,3,44,Lewis Hamilton,Mercedes,1:14.894
3,4,5,Sebastian Vettel,Ferrari,1:14.970
4,5,77,Valtteri Bottas,Mercedes,1:15.160
5,6,7,Kimi Räikkönen,Ferrari,1:15.330
6,7,27,Nico Hulkenberg,Renault,1:15.827
7,8,55,Carlos Sainz,Renault,1:16.084
8,9,16,Charles Leclerc,Sauber Ferrari,1:16.189
9,10,9,Marcus Ericsson,Sauber Ferrari,1:16.513


In [0]:
def format_FastestLap(table_data):
  #FASTEST LAP: 
  Table = pd.DataFrame()
  Pos, No, Driver, Car, Lap, Time_of_day, Time, Avg_Speed = list(), list(), list(), list(), list(), list(), list(), list()

  for line in table_data:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 8:
      Pos.append(temp[0])
      No.append(temp[1]) 
      Driver.append(temp[2]+" "+temp[3])
      Car.append(temp[5])
      Lap.append(temp[6])
      Time_of_day.append(temp[7])
      Time.append(temp[8])
      Avg_Speed.append(temp[9])   
    


  Table["Pos"], Table["No"], Table["Driver"], Table["Car"], Table["Lap"], Table["Time of Day"], Table["Time"], Table["Avg Speed"]  = Pos, No, Driver, Car, Lap, Time_of_day, Time, Avg_Speed
  return Table

In [17]:
# EXAMPLE:
td = get_TableData(yr,i_d,r,tables[2]) 
format_FastestLap(td)

,Pos,No,Driver,Car,Lap,Time of Day,Time,Avg Speed
0,1,77,Valtteri Bottas,Mercedes,65,14:44:54,1:18.741,196.776
1,2,33,Max Verstappen,Red Bull Racing TAG Heuer,66,14:44:46,1:19.186,195.670
2,3,3,Daniel Ricciardo,Red Bull Racing TAG Heuer,57,14:32:09,1:19.462,194.991
3,4,5,Sebastian Vettel,Ferrari,49,14:21:24,1:19.522,194.844
4,5,7,Kimi Räikkönen,Ferrari,19,13:40:00,1:20.334,192.874
5,6,16,Charles Leclerc,Sauber Ferrari,67,14:49:16,1:20.537,192.388
6,7,27,Nico Hulkenberg,Renault,67,14:49:05,1:20.637,192.150
7,8,44,Lewis Hamilton,Mercedes,49,14:21:46,1:20.728,191.933
8,9,8,Romain Grosjean,Haas Ferrari,46,14:19:51,1:21.370,190.419
9,10,20,Kevin Magnussen,Haas Ferrari,45,14:18:20,1:21.874,189.246


In [0]:
def format_RaceResults(table_data):
  #RACE RESULTS:
  Table = pd.DataFrame()
  Pos, No, Driver, Car, Laps, Time, PTS = list(), list(), list(), list(), list(), list(), list()

  for line in table_data:
    elemnts = line.split('\n')
    temp = list()
    for e in elemnts:
      if e != '':
        temp.append(e)
    if len(temp) > 8:
      Pos.append(temp[0])
      No.append(temp[1]) 
      Driver.append(temp[2]+" "+temp[3])
      Car.append(temp[5])
      Laps.append(temp[6])
      Time.append(temp[7])
      PTS.append(temp[8])

  Table["Pos"], Table["No"], Table["Driver"], Table["Car"], Table["Laps"], Table["Time"], Table["PTS"] = Pos, No, Driver, Car, Laps, Time, PTS
  return Table

In [19]:
# EXAMPLE:
td = get_TableData(yr,i_d,r,tables[3]) 
format_RaceResults(td)

,Pos,No,Driver,Car,Laps,Time,PTS
0,1,33,Max Verstappen,Red Bull Racing TAG Heuer,71,1:38:28.851,25
1,2,5,Sebastian Vettel,Ferrari,71,+17.316s,18
2,3,7,Kimi Räikkönen,Ferrari,71,+49.914s,15
3,4,44,Lewis Hamilton,Mercedes,71,+78.738s,12
4,5,77,Valtteri Bottas,Mercedes,70,+1 lap,10
5,6,27,Nico Hulkenberg,Renault,69,+2 laps,8
6,7,16,Charles Leclerc,Sauber Ferrari,69,+2 laps,6
7,8,2,Stoffel Vandoorne,McLaren Renault,69,+2 laps,4
8,9,9,Marcus Ericsson,Sauber Ferrari,69,+2 laps,2
9,10,10,Pierre Gasly,Scuderia Toro Rosso Honda,69,+2 laps,1


In [0]:
def aggregate_Season(year):
  Race_Index = get_RaceIndex(year)
  Race_Drivers = get_Drivers(year)

  RACE_DATAFRAME = pd.DataFrame({"Drivers": Race_Drivers})
  for race in get_Races(year):
    Q_data, S_data, R_data = get_TableData(year,Race_Index,race,"qualifying"), get_TableData(year,Race_Index,race,"starting-grid"), get_TableData(year,Race_Index,race,"race-result")
    Q_df, S_df, R_df = format_Qualifying(Q_data), format_StartingGrid(S_data), format_RaceResults(R_data)
    
    POS_DICT = dict(zip(Race_Drivers,["NO DATA"]*len(Race_Drivers)))
    for row in Q_df.itertuples():
      POS_DICT[row.Driver] = row.Pos
    RACE_DATAFRAME[str(race.capitalize())+str(" Qual")] = RACE_DATAFRAME['Drivers'].apply(lambda d : POS_DICT[d])

    POS_DICT = dict(zip(Race_Drivers,["NO DATA"]*len(Race_Drivers)))
    for row in S_df.itertuples():
      POS_DICT[row.Driver] = row.Pos
    RACE_DATAFRAME[str(race.capitalize())+str(" Start")] = RACE_DATAFRAME['Drivers'].apply(lambda d : POS_DICT[d])

    POS_DICT = dict(zip(Race_Drivers,["NO DATA"]*len(Race_Drivers)))
    for row in R_df.itertuples():
      POS_DICT[row.Driver] = row.Pos
    RACE_DATAFRAME[str(race.capitalize())+str(" Race")] = RACE_DATAFRAME['Drivers'].apply(lambda d : POS_DICT[d])
  return RACE_DATAFRAME

In [21]:
aggregate_Season(2019)

,Drivers,Australia Qual,Australia Start,Australia Race,Bahrain Qual,Bahrain Start,Bahrain Race,China Qual,China Start,China Race,Azerbaijan Qual,Azerbaijan Start,Azerbaijan Race,Spain Qual,Spain Start,Spain Race,Monaco Qual,Monaco Start,Monaco Race,Canada Qual,Canada Start,Canada Race,France Qual,France Start,France Race,Austria Qual,Austria Start,Austria Race,Great britain Qual,Great britain Start,Great britain Race,Germany Qual,Germany Start,Germany Race,Hungary Qual,Hungary Start,Hungary Race,Belgium Qual,Belgium Start,Belgium Race,Italy Qual,Italy Start,Italy Race,Singapore Qual,Singapore Start,Singapore Race,Russia Qual,Russia Start,Russia Race,Japan Qual,Japan Start,Japan Race,Mexico Qual,Mexico Start,Mexico Race,United states Qual,United states Start,United states Race,Brazil Qual,Brazil Start,Brazil Race,Abu dhabi Qual,Abu dhabi Start,Abu dhabi Race
0,Lewis Hamilton,1,1,2,3,3,1,2,2,1,2,2,2,2,2,1,1,1,1,2,2,1,1,1,1,2,4,5,2,2,1,1,1,9,3,3,1,3,3,2,2,2,3,2,2,4,2,2,1,4,4,3,4,3,1,5,5,2,3,3,7,1,1,1
1,Valtteri Bottas,2,2,1,4,4,2,1,1,2,1,1,1,1,1,2,2,2,3,6,6,4,2,2,2,4,3,3,1,1,2,3,3,NC,2,2,8,4,4,3,3,3,2,5,5,5,5,4,2,3,3,1,6,6,3,1,1,1,5,4,NC,2,20,4
2,Max Verstappen,4,4,3,5,5,4,5,5,4,4,4,4,4,4,3,3,3,4,11,9,5,4,4,4,3,2,1,4,4,5,2,2,1,1,1,2,5,5,NC,NC,19,8,4,4,3,4,9,4,5,5,NC,1,4,6,3,3,3,1,1,1,3,2,2
3,Charles Leclerc,5,5,5,1,1,3,4,4,5,9,8,5,5,5,5,16,15,NC,3,3,3,3,3,3,1,1,2,3,3,3,10,10,NC,4,4,4,1,1,1,1,1,1,1,1,2,1,1,3,2,2,6,2,1,4,4,4,4,4,14,18,4,3,3
4,Sebastian Vettel,3,3,4,2,2,5,3,3,3,3,3,3,3,3,4,4,4,2,1,1,2,7,7,5,10,9,4,6,6,16,NC,20,2,5,5,3,2,2,4,4,4,13,3,3,1,3,3,NC,1,1,2,3,2,2,2,2,NC,2,2,17,5,4,5
5,Carlos Sainz,18,18,NC,7,7,19,14,14,14,10,9,7,13,12,8,9,9,6,9,11,11,6,6,6,15,19,8,13,13,6,7,7,5,8,8,5,17,15,NC,7,7,NC,7,7,12,6,5,6,7,7,5,7,7,13,7,7,8,NC,20,3,9,8,10
6,Pierre Gasly,17,17,11,13,13,8,6,6,6,NC,20,NC,6,6,6,5,8,5,5,5,8,9,9,10,9,8,7,5,5,4,4,4,14,6,6,6,16,13,9,15,17,11,12,11,8,11,16,14,9,9,7,10,10,9,10,10,16,7,6,2,12,11,18
7,Alexander Albon,13,13,14,12,12,9,NO DATA,20,10,12,11,11,12,11,11,10,10,8,14,13,NC,11,11,15,13,18,15,9,9,12,17,16,6,12,12,10,14,17,5,8,8,6,6,6,6,19,20,5,6,6,4,5,5,5,6,6,5,6,5,14,6,5,6
8,Daniel Ricciardo,12,12,NC,11,10,18,7,7,7,11,10,NC,10,13,12,7,6,9,4,4,6,8,8,11,14,12,12,7,7,7,13,13,NC,18,20,14,6,10,14,5,5,4,NC,20,14,10,10,NC,16,16,DQ,13,13,8,9,9,6,12,11,6,8,7,11
9,Sergio Perez,10,10,13,14,14,10,12,12,8,5,5,6,15,15,15,17,16,12,16,15,12,14,14,12,16,13,11,15,15,17,8,8,NC,17,16,11,9,7,6,17,18,7,10,15,NC,12,11,7,17,17,8,11,11,7,19,20,10,15,15,9,11,10,7


In [0]:
def aggregate_Season_V2(year):
  Race_Index = get_RaceIndex(year)
  Race_Drivers = get_Drivers(year)

  RACE_DATAFRAME = pd.DataFrame()
  for race in get_Races(year):
    TEMP = pd.DataFrame({"Drivers": Race_Drivers})
    TEMP["Race"] = list(np.repeat(race.capitalize(),len(TEMP['Drivers'])))
    Q_data, S_data, R_data = get_TableData(year,Race_Index,race,"qualifying"), get_TableData(year,Race_Index,race,"starting-grid"), get_TableData(year,Race_Index,race,"race-result")
    Q_df, S_df, R_df = format_Qualifying(Q_data), format_StartingGrid(S_data), format_RaceResults(R_data)
    
    POS_DICT = dict(zip(Race_Drivers,["NO DATA"]*len(Race_Drivers)))
    for row in Q_df.itertuples():
      POS_DICT[row.Driver] = row.Pos
    TEMP["Qualification"] = TEMP['Drivers'].apply(lambda d : POS_DICT[d])

    POS_DICT = dict(zip(Race_Drivers,["NO DATA"]*len(Race_Drivers)))
    for row in S_df.itertuples():
      POS_DICT[row.Driver] = row.Pos
    TEMP["Start"] = TEMP['Drivers'].apply(lambda d : POS_DICT[d])

    POS_DICT = dict(zip(Race_Drivers,["NO DATA"]*len(Race_Drivers)))
    for row in R_df.itertuples():
      POS_DICT[row.Driver] = row.Pos
    TEMP["Result"] = TEMP['Drivers'].apply(lambda d : POS_DICT[d])

    RACE_DATAFRAME = RACE_DATAFRAME.append(TEMP)
  return RACE_DATAFRAME

In [23]:
a_S_2019 = aggregate_Season_V2(2019)
a_S_2019

,Drivers,Race,Qualification,Start,Result
0,Lewis Hamilton,Australia,1,1,2
1,Valtteri Bottas,Australia,2,2,1
2,Max Verstappen,Australia,4,4,3
3,Charles Leclerc,Australia,5,5,5
4,Sebastian Vettel,Australia,3,3,4
...,...,...,...,...,...
15,Kevin Magnussen,Abu dhabi,15,14,14
16,Antonio Giovinazzi,Abu dhabi,17,16,16
17,Romain Grosjean,Abu dhabi,16,15,15
18,Robert Kubica,Abu dhabi,20,19,19


## Scoring Functions:

https://fantasy.formula1.com/points-scoring

In [0]:
def calculate_QualifyingPoints(Qual_DataFrame, Start_DataFrame, Race_DataFrame, Points_Dict):
  first_teammate = set()
  for row in Qual_DataFrame.itertuples():
    Points = 0 
    Points_driveronly = 0 
    
    if row.Pos == "DQ" or row.Pos == "NC":
      if row.Driver in list(Race_DataFrame["Driver"]) or row.Driver in list(Start_DataFrame["Driver"]):
        Points += 1 #If Stewards let Driver Race then give 1 point
      else:
        print(row.Pos,row.Driver,"CHECK DQ or NC")
        Points += -10
        Points_driveronly += -5
    elif row.Q3 != " ":
      Points += 3
    elif row.Q2 != " ":
      Points += 2
    else:
      print(row.Pos)
      
    try:
      not_NC = int(row.Pos)
      if not_NC < 11:
        Points += 11 - int(row.Pos) 
      else:
        Points += 1
    except:
      pass
    
    if row.Car not in first_teammate:
      Points_driveronly += 2
      first_teammate.add(row.Car)

    Points_Dict[row.Driver] += Points + Points_driveronly
    Points_Dict[row.Car] += Points
  return Points_Dict

In [0]:
def calculate_RacePoints(Start_DataFrame, Race_DataFrame, FLap_DataFrame, Points_Dict):
  driver_startgrid = dict(zip(Start_DataFrame["Driver"],Start_DataFrame["Pos"]))
  first_teammate = set()
  Finishing_Position_Bonuses = [25,18,15,12,10,8,6,4,2,1]
  for row in Race_DataFrame.itertuples():
    Points = 0 
    Points_driveronly = 0 
    
    if row.Time != "DNF":
      Points += 1

    if row.Car not in first_teammate:
      Points_driveronly += 3
      first_teammate.add(row.Car)
    
    if row.Pos == "DQ":
      Points_driveronly += -20
    elif row.Pos == "NC":
      Points_driveronly += -15
    else:
      finish_pos = int(row.Pos)
      try:
        start_pos = int(driver_startgrid[row.Driver])
      except:
        start_pos = len(driver_startgrid) + 1 #If in Results but not listed on Startgrid, set postion to last
        print(row.Pos,row.Driver,"CHECK started from Pit?")
      gain_loss =  start_pos - finish_pos
      if gain_loss > 0:
        Points += np.minimum(10,gain_loss*2)
      else:
        if start_pos < 11:
          Points += np.maximum(-10, gain_loss*2)
        else:
          Points += np.maximum(-5, gain_loss)

      if finish_pos < 11:
        Points += Finishing_Position_Bonuses[finish_pos-1]
      
      #print(row.Driver,Points_driveronly)
      #print(row.Driver,Points)

    Points_Dict[row.Driver] += Points + Points_driveronly
    Points_Dict[row.Car] += Points

  Points_Dict[FLap_DataFrame.iloc[0].Driver] += 5
  return Points_Dict

In [0]:
def aggregate_SeasonPoints(year):
  Race_Index = get_RaceIndex(year)
  Race_Drivers = get_Drivers(year)
  Race_Teams = get_Teams(year)

  RACE_POINT_DATAFRAME = pd.DataFrame({"Drivers/Teams": Race_Drivers + Race_Teams})
  for race in get_Races(year):
    print(race.capitalize()) 
    Q_data, S_data, R_data, F_data = get_TableData(year,Race_Index,race,"qualifying"), get_TableData(year,Race_Index,race,"starting-grid"), get_TableData(year,Race_Index,race,"race-result"), get_TableData(year,Race_Index,race,"fastest-laps")
    Q_df, S_df, R_df, F_df = format_Qualifying(Q_data), format_StartingGrid(S_data), format_RaceResults(R_data), format_FastestLap(F_data)
    #print("Qualifying")
    POINTS_DICT = dict(zip(Race_Drivers + Race_Teams,np.repeat(0,len(Race_Drivers + Race_Teams))))
    POINTS_DICT = calculate_QualifyingPoints(Q_df, S_df, R_df, POINTS_DICT)
    RACE_POINT_DATAFRAME[str(race.capitalize())+str(" Qual")] = RACE_POINT_DATAFRAME['Drivers/Teams'].apply(lambda d : POINTS_DICT[d])
    #print("Race")
    POINTS_DICT = dict(zip(Race_Drivers + Race_Teams,np.repeat(0,len(Race_Drivers + Race_Teams))))
    POINTS_DICT = calculate_RacePoints(S_df, R_df, F_df, POINTS_DICT)
    RACE_POINT_DATAFRAME[str(race.capitalize())+str(" Race")] = RACE_POINT_DATAFRAME['Drivers/Teams'].apply(lambda d : POINTS_DICT[d])
    #print("--------------------")
  return RACE_POINT_DATAFRAME

In [28]:
PTS_2019 = aggregate_SeasonPoints(2019)
PTS_2019

Australia
16
17
18
19
20
Bahrain
16
17
18
19
20
China
16
17
18
Azerbaijan
14
15
16
17
18
Spain
16
17
18
19
20
Monaco
16
17
18
19
20
Canada
16
17
18
19
20
France
16
17
18
19
20
Austria
16
17
18
19
20
Great britain
16
17
18
19
20
Germany
16
17
18
19
Hungary
16
17
18
19
20
Belgium
15
16
17
18
19
Italy
16
17
18
19
Singapore
15
16
17
18
19
Russia
16
17
18
19
Japan
16
17
18
17 Robert Kubica CHECK started from Pit?
Mexico
16
17
18
19
20
United states
16
17
18
19
20
Brazil
16
17
18
19
Abu dhabi
16
17
18
19
20


,Drivers/Teams,Australia Qual,Australia Race,Bahrain Qual,Bahrain Race,China Qual,China Race,Azerbaijan Qual,Azerbaijan Race,Spain Qual,Spain Race,Monaco Qual,Monaco Race,Canada Qual,Canada Race,France Qual,France Race,Austria Qual,Austria Race,Great britain Qual,Great britain Race,Germany Qual,Germany Race,Hungary Qual,Hungary Race,Belgium Qual,Belgium Race,Italy Qual,Italy Race,Singapore Qual,Singapore Race,Russia Qual,Russia Race,Japan Qual,Japan Race,Mexico Qual,Mexico Race,United states Qual,United states Race,Brazil Qual,Brazil Race,Abu dhabi Qual,Abu dhabi Race
0,Lewis Hamilton,15,17,13,33,12,31,12,19,12,36,15,29,14,31,15,29,14,9,12,36,15,-4,11,33,13,24,14,19,14,12,14,36,10,23,12,33,9,25,13,2,15,34
1,Valtteri Bottas,12,36,10,23,15,17,15,29,15,17,12,14,8,22,12,19,10,19,15,17,11,-15,14,-5,10,18,11,24,9,11,9,23,13,33,8,22,15,29,9,-10,12,23
2,Max Verstappen,12,21,11,18,11,18,12,16,12,21,13,14,3,22,12,16,13,36,12,9,14,36,15,25,11,-15,1,15,12,21,12,26,11,-15,15,5,13,19,15,29,13,22
3,Charles Leclerc,9,11,15,20,10,9,4,22,9,11,1,-15,11,16,13,19,15,20,13,19,5,-15,12,13,15,29,15,29,15,17,15,15,12,1,14,12,10,21,10,-4,12,19
4,Sebastian Vettel,13,14,12,5,13,19,13,19,13,14,12,26,15,20,7,20,3,23,8,-9,1,32,9,23,12,14,10,-9,11,33,11,-15,15,20,11,22,14,-15,14,-7,9,9
5,Carlos Sainz,1,-15,9,-10,5,4,3,14,3,16,7,18,5,4,8,12,3,15,3,22,9,18,6,20,1,-15,9,-15,9,-9,10,10,9,18,9,-6,9,3,1,29,5,-2
6,Pierre Gasly,1,11,3,15,8,14,1,-15,8,9,9,22,11,-1,5,0,5,9,9,18,10,-10,8,9,3,11,3,14,5,14,5,5,7,14,4,8,6,-10,9,30,5,-4
7,Alexander Albon,5,0,5,12,0,15,3,4,3,1,4,9,3,-15,5,-3,5,10,7,-5,1,19,5,9,3,24,8,16,8,9,1,21,8,20,9,14,8,13,8,-9,8,7
8,Daniel Ricciardo,3,-15,5,-10,9,10,5,-15,6,6,9,0,12,8,8,-5,3,4,9,10,3,-15,1,11,10,-7,11,18,3,11,4,-15,1,-16,3,18,7,18,5,22,8,-4
9,Sergio Perez,6,-5,5,13,5,16,11,10,5,4,3,12,3,7,5,8,3,8,5,-1,8,-15,3,14,7,14,1,20,5,-15,5,18,1,18,5,18,1,15,5,16,5,16


In [0]:
def aggregate_SeasonPoints_V2(year):
  Race_Index = get_RaceIndex(year)
  Race_Drivers = get_Drivers(year)
  Race_Teams = get_Teams(year)

  RACE_POINT_DATAFRAME = pd.DataFrame()
  for race in get_Races(year):
    TEMP = pd.DataFrame({"Drivers/Teams": Race_Drivers + Race_Teams})
    TEMP["Venue"] = list(np.repeat(race.capitalize(),len(TEMP['Drivers/Teams'])))
    TEMP["Type"] = list(np.repeat("Driver",len(Race_Drivers)))+list(np.repeat("Constructor",len(Race_Teams)))
    print(race.capitalize()) 
    Q_data, S_data, R_data, F_data = get_TableData(year,Race_Index,race,"qualifying"), get_TableData(year,Race_Index,race,"starting-grid"), get_TableData(year,Race_Index,race,"race-result"), get_TableData(year,Race_Index,race,"fastest-laps")
    Q_df, S_df, R_df, F_df = format_Qualifying(Q_data), format_StartingGrid(S_data), format_RaceResults(R_data), format_FastestLap(F_data)
    print("Qualifying")
    POINTS_DICT = dict(zip(Race_Drivers + Race_Teams,np.repeat(0,len(Race_Drivers + Race_Teams))))
    POINTS_DICT = calculate_QualifyingPoints(Q_df, S_df, R_df, POINTS_DICT)
    TEMP["Qualifying"] = TEMP['Drivers/Teams'].apply(lambda d : POINTS_DICT[d])
    print("Race")
    POINTS_DICT = dict(zip(Race_Drivers + Race_Teams,np.repeat(0,len(Race_Drivers + Race_Teams))))
    POINTS_DICT = calculate_RacePoints(S_df, R_df, F_df, POINTS_DICT)
    TEMP["Race"] = TEMP['Drivers/Teams'].apply(lambda d : POINTS_DICT[d])

    RACE_POINT_DATAFRAME = RACE_POINT_DATAFRAME.append(TEMP)

    print("--------------------")
  return RACE_POINT_DATAFRAME

In [30]:
PTS_2019_V2 = aggregate_SeasonPoints_V2(2019)
PTS_2019_V2

Australia
Qualifying
16
17
18
19
20
Race
--------------------
Bahrain
Qualifying
16
17
18
19
20
Race
--------------------
China
Qualifying
16
17
18
Race
--------------------
Azerbaijan
Qualifying
14
15
16
17
18
Race
--------------------
Spain
Qualifying
16
17
18
19
20
Race
--------------------
Monaco
Qualifying
16
17
18
19
20
Race
--------------------
Canada
Qualifying
16
17
18
19
20
Race
--------------------
France
Qualifying
16
17
18
19
20
Race
--------------------
Austria
Qualifying
16
17
18
19
20
Race
--------------------
Great britain
Qualifying
16
17
18
19
20
Race
--------------------
Germany
Qualifying
16
17
18
19
Race
--------------------
Hungary
Qualifying
16
17
18
19
20
Race
--------------------
Belgium
Qualifying
15
16
17
18
19
Race
--------------------
Italy
Qualifying
16
17
18
19
Race
--------------------
Singapore
Qualifying
15
16
17
18
19
Race
--------------------
Russia
Qualifying
16
17
18
19
Race
--------------------
Japan
Qualifying
16
17
18
Race
17 Robert Kubica CHEC

,Drivers/Teams,Venue,Type,Qualifying,Race
0,Lewis Hamilton,Australia,Driver,15,17
1,Valtteri Bottas,Australia,Driver,12,36
2,Max Verstappen,Australia,Driver,12,21
3,Charles Leclerc,Australia,Driver,9,11
4,Sebastian Vettel,Australia,Driver,13,14
...,...,...,...,...,...
25,Scuderia Toro Rosso Honda,Abu dhabi,Constructor,6,7
26,Racing Point BWT Mercedes,Abu dhabi,Constructor,6,13
27,Alfa Romeo Racing Ferrari,Abu dhabi,Constructor,2,10
28,Haas Ferrari,Abu dhabi,Constructor,4,2


## Picking 2020 Team based on 2019:
https://fantasy.formula1.com/game-rules

In [31]:
year = 2019
Race_Drivers = get_Drivers(year)
Race_Teams = get_Teams(year)

Fantasy_Data = pd.DataFrame()
prices_0 = [31.3, 28.4, 26.1, 24.2, 21.8, 15.5, 10.4, 20.3, 14.1, 9.3, 11.5, 10.3, 9.9, np.NaN, 7.9, 8.2, 7.7, 6, np.NaN, 5.9, 32.2, 27.4, 24.6, 14.9, 12.6, 13.2, 10.1, 8.7, 7.8, 6.5]

Fantasy_Data["Drivers/Teams"] = list(Race_Drivers) + list(Race_Teams)
Fantasy_Data["Price"] = prices_0
Fantasy_Data["Type"] = list(np.repeat("Driver",len(Race_Drivers)))+list(np.repeat("Constructor",len(Race_Teams)))
Fantasy_Data["Turbo"] = ["Turbo" if Fantasy_Data["Price"][i] < 20.0 and Fantasy_Data["Type"][i] == "Driver"  else "Not" for i in range(Fantasy_Data.shape[0])]

Fantasy_Data.head()

,Drivers/Teams,Price,Type,Turbo
0,Lewis Hamilton,31.3,Driver,Not
1,Valtteri Bottas,28.4,Driver,Not
2,Max Verstappen,26.1,Driver,Not
3,Charles Leclerc,24.2,Driver,Not
4,Sebastian Vettel,21.8,Driver,Not


In [32]:
Season_Points = aggregate_SeasonPoints_V2(year)

Australia
Qualifying
16
17
18
19
20
Race
--------------------
Bahrain
Qualifying
16
17
18
19
20
Race
--------------------
China
Qualifying
16
17
18
Race
--------------------
Azerbaijan
Qualifying
14
15
16
17
18
Race
--------------------
Spain
Qualifying
16
17
18
19
20
Race
--------------------
Monaco
Qualifying
16
17
18
19
20
Race
--------------------
Canada
Qualifying
16
17
18
19
20
Race
--------------------
France
Qualifying
16
17
18
19
20
Race
--------------------
Austria
Qualifying
16
17
18
19
20
Race
--------------------
Great britain
Qualifying
16
17
18
19
20
Race
--------------------
Germany
Qualifying
16
17
18
19
Race
--------------------
Hungary
Qualifying
16
17
18
19
20
Race
--------------------
Belgium
Qualifying
15
16
17
18
19
Race
--------------------
Italy
Qualifying
16
17
18
19
Race
--------------------
Singapore
Qualifying
15
16
17
18
19
Race
--------------------
Russia
Qualifying
16
17
18
19
Race
--------------------
Japan
Qualifying
16
17
18
Race
17 Robert Kubica CHEC

In [0]:
def assign_Points(yr, fantasy_df, pts_df, theta = "sum", nLast_races = None, add_random = True):
  points = []
  for dv_cr in fantasy_df["Drivers/Teams"]:
    season_results = pts_df[pts_df["Drivers/Teams"]==dv_cr]
    pts_vec = list(season_results.Qualifying + season_results.Race)[10:]
    n = len(pts_vec)

    if add_random == True:
      pts_vec = np.random.choice(pts_vec,n,replace=True)

    if theta == "sum":
      points.append(np.sum(pts_vec))
    elif theta == "mean":
      points.append(np.mean(pts_vec))
    elif theta == "median":
      points.append(np.median(pts_vec))
    else:
      print("ERROR: set 'theta' = 'sum', 'mean' or 'median'")
  data = fantasy_df.copy()
  data["Points"] = points
  data = data.dropna()
  data = data.reset_index(drop=True)
  return data

In [133]:
data = assign_Points(year, Fantasy_Data, Season_Points, theta = "sum", nLast_races = None, add_random = False)
data

,Drivers/Teams,Price,Type,Turbo,Points
0,Lewis Hamilton,31.3,Driver,Not,377
1,Valtteri Bottas,28.4,Driver,Not,274
2,Max Verstappen,26.1,Driver,Not,300
3,Charles Leclerc,24.2,Driver,Not,272
4,Sebastian Vettel,21.8,Driver,Not,224
5,Carlos Sainz,15.5,Driver,Turbo,128
6,Pierre Gasly,10.4,Driver,Turbo,146
7,Alexander Albon,20.3,Driver,Not,210
8,Daniel Ricciardo,14.1,Driver,Turbo,97
9,Sergio Perez,9.3,Driver,Turbo,165


In [129]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

def main():
  ## Create Data:
  data = assign_Points(year, Fantasy_Data, Season_Points, theta = "sum", nLast_races = None, add_random = True)
  solver = pywraplp.Solver('simple_mip_program',
                            pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
  x = {}
  L = data.shape[0]
  turbo_map = {}
  for j in range(data.shape[0]):
    x[j] = solver.IntVar(0, 1, 'x[%s]' % data["Drivers/Teams"][j])
    if data["Turbo"][j] == "Turbo":
      x[L] = solver.IntVar(0, 1, 't[%s]' % data["Drivers/Teams"][j])
      turbo_map[L] = j 
      L += 1
  
  constraint_expr = [data["Price"][j] * x[j] for j in range(data.shape[0])]
  solver.Add(sum(constraint_expr) <= 100.0)

  constraint_expr = [x[j] for j in range(data.shape[0])]
  solver.Add(sum(constraint_expr) <= 6)

  drivers, constructors = [], []
  for j in range(data.shape[0]):
    if data["Type"][j] == "Driver":
      drivers.append(x[j])
    elif data["Type"][j] == "Constructor":
      constructors.append(x[j])
    else:
      print("ERROR: Seat is either driver or constructor.")

  solver.Add(sum(drivers) == 5)
  solver.Add(sum(constructors) == 1)

  ## Turbo:
  constraint_expr = [x[j] for j in range(L-len(turbo_map),L)]
  solver.Add(sum(constraint_expr) <= 1)

  for t in turbo_map:
    solver.Add(x[t] - x[turbo_map[t]]  <= 0)

  objective = solver.Objective()
  for j in range(L):
    if j < data.shape[0]:
      objective.SetCoefficient(x[j], float(data['Points'][j]))
    else:
      objective.SetCoefficient(x[j], float(data['Points'][turbo_map[j]]))
  objective.SetMaximization()
  
  status = solver.Solve()

  if status == pywraplp.Solver.OPTIMAL:
    print('Total Points =', int(solver.Objective().Value()))
    Cost = []
    data["Solution"] = 0.0
    for j in range(L):
      if x[j].solution_value() > 0:
        if j < data.shape[0]:
          print(x[j].name(), ' = ',data["Points"][j]," $",data["Price"][j])
          Cost.append(data["Price"][j])
          data.loc[j, 'Solution'] = 1
        else:
          print(x[j].name(), ' = ',data["Points"][turbo_map[j]],"<- TURBO")
          data.loc[turbo_map[j], 'Solution'] = 2
    print("Total Cost $",sum(Cost))
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
  else:
    print('The problem does not have an optimal solution.')
  return data
if __name__ == '__main__':
  main()

Total Points = 3547
x[Lewis Hamilton]  =  827  $ 31.3
x[Pierre Gasly]  =  247  $ 10.4
x[Sergio Perez]  =  321  $ 9.3
x[Daniil Kvyat]  =  291  $ 9.9
x[George Russell]  =  221  $ 5.9
x[Mercedes]  =  1319  $ 32.2
t[Sergio Perez]  =  321 <- TURBO
Total Cost $ 99.0

Problem solved in 46.000000 milliseconds
Problem solved in 5 iterations
Problem solved in 0 branch-and-bound nodes


In [0]:
runs = 250
all_runs = list()
for _ in range(runs):
  all_runs.append(tuple(main()["Solution"]))

In [135]:
count = dict()
check_seen = set()
for combo in all_runs:
  if combo not in check_seen:
    check_seen.add(combo)
    count[combo] = 1
  else:
    count[combo] += 1

from collections import Counter 

k = Counter(count) 
high = k.most_common(3)  

for i in range(3):
  print(high[i][1],"/",runs)
  data['select'] = list(high[i][0])
  print( data[data['select'] > 0])
  print("\n")

11 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver    Not     377     1.0
6     Pierre Gasly   10.4       Driver  Turbo     146     1.0
9     Sergio Perez    9.3       Driver  Turbo     165     2.0
12    Daniil Kvyat    9.9       Driver  Turbo     118     1.0
17  George Russell    5.9       Driver  Turbo      68     1.0
18        Mercedes   32.2  Constructor    Not     601     1.0


8 / 250
      Drivers/Teams  Price         Type  Turbo  Points  select
0    Lewis Hamilton   31.3       Driver    Not     377     1.0
6      Pierre Gasly   10.4       Driver  Turbo     146     1.0
9      Sergio Perez    9.3       Driver  Turbo     165     2.0
13     Lance Stroll    7.9       Driver  Turbo      90     1.0
14  Kevin Magnussen    8.2       Driver  Turbo      75     1.0
18         Mercedes   32.2  Constructor    Not     601     1.0


6 / 250
            Drivers/Teams  Price         Type  Turbo  Points  select
2          Max Verstappen  

In [131]:
count = dict()
check_seen = set()
for combo in all_runs:
  if combo not in check_seen:
    check_seen.add(combo)
    count[combo] = 1
  else:
    count[combo] += 1

from collections import Counter 

k = Counter(count) 
high = k.most_common(3)  

for i in range(3):
  print(high[i][1],"/",runs)
  data['select'] = list(high[i][0])
  print( data[data['select'] > 0])
  print("\n")

56 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver    Not     781     1.0
6     Pierre Gasly   10.4       Driver  Turbo     288     2.0
9     Sergio Perez    9.3       Driver  Turbo     288     1.0
11  Kimi Räikkönen   10.3       Driver  Turbo     242     1.0
17  George Russell    5.9       Driver  Turbo     185     1.0
18        Mercedes   32.2  Constructor    Not    1272     1.0


47 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver    Not     781     1.0
6     Pierre Gasly   10.4       Driver  Turbo     288     1.0
9     Sergio Perez    9.3       Driver  Turbo     288     2.0
11  Kimi Räikkönen   10.3       Driver  Turbo     242     1.0
17  George Russell    5.9       Driver  Turbo     185     1.0
18        Mercedes   32.2  Constructor    Not    1272     1.0


29 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver 

In [125]:
count = dict()
check_seen = set()
for combo in all_runs:
  if combo not in check_seen:
    check_seen.add(combo)
    count[combo] = 1
  else:
    count[combo] += 1

from collections import Counter 

k = Counter(count) 
high = k.most_common(3)  

for i in range(3):
  print(high[i][1],"/",runs)
  data['select'] = list(high[i][0])
  print( data[data['select'] > 0])
  print("\n")

49 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver    Not    43.0     1.0
6     Pierre Gasly   10.4       Driver  Turbo    14.0     1.0
9     Sergio Perez    9.3       Driver  Turbo    17.0     2.0
11  Kimi Räikkönen   10.3       Driver  Turbo    13.0     1.0
17  George Russell    5.9       Driver  Turbo    11.0     1.0
18        Mercedes   32.2  Constructor    Not    70.0     1.0


38 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver    Not    43.0     1.0
6     Pierre Gasly   10.4       Driver  Turbo    14.0     1.0
9     Sergio Perez    9.3       Driver  Turbo    17.0     2.0
12    Daniil Kvyat    9.9       Driver  Turbo    13.0     1.0
17  George Russell    5.9       Driver  Turbo    11.0     1.0
18        Mercedes   32.2  Constructor    Not    70.0     1.0


16 / 250
     Drivers/Teams  Price         Type  Turbo  Points  select
0   Lewis Hamilton   31.3       Driver 

## Saving Data to Google Sheets:

In [0]:
from google.colab import auth
auth.authenticate_user()
from gspread_dataframe import set_with_dataframe

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
## 2019 Driver Places:

#gc.create('F1_2019_Places')

worksheet = gc.open('F1_2019_Places').sheet1
set_with_dataframe(worksheet, a_S_2019,include_index=False)

In [0]:
## 2019 Driver & Teams Points:

#gc.create('F1_2019_Pts')

worksheet = gc.open('F1_2019_Pts').sheet1
set_with_dataframe(worksheet, PTS_2019,include_index=False)

In [0]:
## 2019 Driver & Teams Points (gathered by Race):

#gc.create('F1_2019_Pts_v2')

worksheet = gc.open('F1_2019_Pts_v2').sheet1
set_with_dataframe(worksheet, PTS_2019_V2,include_index=False)

In [0]:
## 2019 Driver Places merge Driver & Teams Points / Prices (gathered by Race):

a_S_2019["Key"] = a_S_2019["Drivers"]+a_S_2019["Race"]
PTS_2019_V2["Key"] = PTS_2019_V2["Drivers/Teams"]+PTS_2019_V2["Venue"]

Tableau_df = PTS_2019_V2.merge(a_S_2019[["Key","Qualification","Start","Result"]],how='left',on='Key')
Tableau_df.head()

#gc.create('F1_2019_Pos_Pts')

worksheet = gc.open('F1_2019_Pos_Pts').sheet1
set_with_dataframe(worksheet, Tableau_df.merge(Fantasy_Data[["Drivers/Teams","Price","Turbo"]],how="left",on="Drivers/Teams"),include_index=False)

## Extra: 
OR Tools & Pulp Version w/o Turbo constraint & Bootstrapping interval...

In [262]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

def main():
  # Create the mip solver with the CBC backend.
  solver = pywraplp.Solver('simple_mip_program',
                            pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
  x = {}
  for j in range(data.shape[0]):
    x[j] = solver.IntVar(0, 1, 'x[%s]' % data["Drivers/Teams"][j])
  #print('Number of variables =', solver.NumVariables())

  constraint_expr = [data["Price"][j] * x[j] for j in range(data.shape[0])]
  solver.Add(sum(constraint_expr) <= 100.0)

  constraint_expr = [x[j] for j in range(data.shape[0])]
  solver.Add(sum(constraint_expr) <= 6)

  drivers, constructors = [], []
  for j in range(data.shape[0]):
    if data["Type"][j] == "Driver":
      drivers.append(x[j])
    elif data["Type"][j] == "Constructor":
      constructors.append(x[j])
    else:
      print("ERROR: Seat is either driver or constructor.")

  solver.Add(sum(drivers) == 5)
  solver.Add(sum(constructors) == 1)

  objective = solver.Objective()
  for j in range(data.shape[0]):
    objective.SetCoefficient(x[j], float(data['Points'][j]))
  objective.SetMaximization()
    
  status = solver.Solve()
  
  if status == pywraplp.Solver.OPTIMAL:
    print('Total Points =', int(solver.Objective().Value()))
    Cost = []
    for j in range(data.shape[0]):
      if x[j].solution_value() > 0:
        #print(x[j].name(), ' = ',data["Drivers/Teams"][j]," - ",x[j].solution_value())
        print(x[j].name(), ' = ',data["Points"][j]," $",data["Price"][j])
        Cost.append(data["Price"][j])
    print("Total Cost $",sum(Cost))
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
  else:
    print('The problem does not have an optimal solution.')


if __name__ == '__main__':
  main()

Total Points = 3056
x[Lewis Hamilton]  =  781  $ 31.3
x[Pierre Gasly]  =  288  $ 10.4
x[Sergio Perez]  =  288  $ 9.3
x[Kimi Räikkönen]  =  242  $ 10.3
x[George Russell]  =  185  $ 5.9
x[Mercedes]  =  1272  $ 32.2
Total Cost $ 99.4

Problem solved in 17.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes


In [198]:
#!pip3 install pulp
from pulp import *

seat = [str(i) for i in range(data.shape[0])]
point = {str(i): data['Points'][i] for i in range(data.shape[0])} 
price = {str(i): data['Price'][i] for i in range(data.shape[0])}
drv = {str(i): 1 if data['Type'][i] == 'Driver' else 0 for i in range(data.shape[0])}
cr = {str(i): 1 if data['Type'][i] == 'Constructor' else 0 for i in range(data.shape[0])}

xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy F1",LpMaximize)
seat_vars = LpVariable.dicts("Seats",seat,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*seat_vars[i] for i in seat]), "Total Cost"

# constraint
prob += lpSum([seat_vars[i] for i in seat]) == 6, "Total 6 Spots"
prob += lpSum([price[i] * seat_vars[i] for i in seat]) <= 100.0, "Total Cost"
prob += lpSum([cr[i] * seat_vars[i] for i in seat]) == 1, "Only 1 GK"
prob += lpSum([drv[i] * seat_vars[i] for i in seat]) == 5, "Less than 5 Drivers"

# solve
status = prob.solve()

## Output: 
print("Status:", LpStatus[prob.status])

selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

O = data[data['integer_programming'] == 1.0]

print("$",O["Price"].sum())
print(O)

Status: Optimal
$ 99.4
     Drivers/Teams  Price         Type  Turbo  Points  integer_programming
0   Lewis Hamilton   31.3       Driver    Not     682                  1.0
6     Pierre Gasly   10.4       Driver  Turbo     265                  1.0
9     Sergio Perez    9.3       Driver  Turbo     244                  1.0
11  Kimi Räikkönen   10.3       Driver  Turbo     297                  1.0
17  George Russell    5.9       Driver  Turbo     216                  1.0
18        Mercedes   32.2  Constructor    Not    1302                  1.0


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [199]:
# BOOTSTRAP INTERVALS:
B = 10000
for driver in get_Drivers(2019):
  driver_results = PTS_2019_V2[PTS_2019_V2["Drivers/Teams"]==driver]
  pts_vec = list(driver_results.Qualifying + driver_results.Race)
  n, Theta = len(pts_vec), np.median(pts_vec)
  boot_samples = np.random.choice(pts_vec,n*B,replace=True).reshape((B,n))
  boot_statistics = np.median(boot_samples, axis=1)
  deltas = np.sort(boot_statistics) - Theta
  Upper = Theta + np.quantile(deltas, 0.90)
  Lower = Theta + np.quantile(deltas, 0.10)
  Range = Upper - Lower
  print(driver,Lower,Theta,Upper,Range)

Lewis Hamilton 33.0 43.0 44.0 11.0
Valtteri Bottas 30.0 32.0 32.0 2.0
Max Verstappen 28.0 29.0 33.0 5.0
Charles Leclerc 25.0 27.0 31.0 6.0
Sebastian Vettel 26.0 27.0 32.0 6.0
Carlos Sainz 9.0 17.0 20.0 11.0
Pierre Gasly 12.0 14.0 17.0 5.0
Alexander Albon 13.0 15.0 17.0 4.0
Daniel Ricciardo 3.0 9.0 14.0 11.0
Sergio Perez 13.0 17.0 21.0 8.0
Lando Norris 4.0 8.0 12.0 8.0
Kimi Räikkönen 9.0 13.0 17.0 8.0
Daniil Kvyat 9.0 13.0 14.0 5.0
Nico Hulkenberg 6.0 8.0 10.0 4.0
Lance Stroll 8.0 10.0 11.0 3.0
Kevin Magnussen 4.0 7.0 9.0 5.0
Antonio Giovinazzi 3.0 4.0 8.0 5.0
Romain Grosjean -4.0 2.0 4.0 8.0
Robert Kubica 4.0 6.0 8.0 4.0
George Russell 9.0 11.0 11.0 2.0
